In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


# Notebook Available here : 

https://github.com/CenturyGhost/kaggle_competition_healh/blob/main/main.ipynb

In [2]:
# Core data science and ML libraries
!pip install numpy pandas matplotlib seaborn scikit-learn
!pip install keras-tuner optuna
!pip install numba termcolor

# PySpark installation
!pip install pyspark==3.5.0

# Additional dependencies that might be needed
!pip install tensorflow[and-cuda]
!pip install tensorflow-probability
!pip install tensorflow-datasets
!pip install tensorflow-hub

!pip install termcolor

!pip install ctypes

!pip install tensorflow-mixed-precision

!pip install pyspark-dist-keras

!pip install scikit-learn

# Verify TensorFlow installation and GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=300bb649dce59a31a7976c2fd9595425159d626568ae20411cef7633466625e2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 69.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 43.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 37.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Set mixed precision policy to improve performance
# Mixed precision uses both 16-bit and 32-bit floating point types
# This can speed up training and reduce memory usage on compatible hardware
policy = Policy('bfloat16')
set_global_policy(policy)

NameError: name 'Policy' is not defined

In [4]:
mixed_precision.set_global_policy('bfloat16')

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

NameError: name 'mixed_precision' is not defined

In [5]:
# Initialize Spark session for the master node
# 1 - Use all available local cores for the master node
# 2 - Set the application name for the master node
# 3 - Configure the driver and executor memory and GPU settings
import subprocess
import json

# Configure Spark with GPU settings for RTX 4090 (24GB) and A5000 (24GB)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ERROR 418 - I'M A TEAPOT") \
    .config("spark.driver.memory", "50g") \
    .config("spark.executor.memory", "50g") \
    .config("spark.driver.maxResultSize", "50g") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.rapids.memory.pinnedPool.size", "24G") \
    .config("spark.rapids.sql.concurrentGpuTasks", "2") \
    .config("spark.rapids.memory.gpu.pooling.enabled", "true") \
    .config("spark.rapids.memory.gpu.allocFraction", "0.95") \
    .config("spark.rapids.sql.explain", "ALL") \
    .config("spark.executor.resource.gpu.amount", "2") \
    .config("spark.task.resource.gpu.amount", "0.25") \
    .config("spark.rapids.sql.incompatibleOps.enabled", "true") \
    .config("spark.rapids.memory.host.spillStorageSize", "64G") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.rapids.memory.gpu.maxAllocFraction", "0.95") \
    .config("spark.rapids.sql.batchSizeBytes", "512M") \
    .config("spark.rapids.sql.reader.batchSizeRows", "100000") \
    .config("spark.rapids.sql.variableRowGroupSize.enabled", "true") \
    .getOrCreate()

gpus = tf.config.list_logical_devices('GPU')
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
      b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)

# Set log level to INFO
spark.sparkContext.setLogLevel("INFO")
# Configure logging
log4jLogger = spark._jvm.org.apache.log4j
logger = log4jLogger.LogManager.getLogger(__name__)

def custom_logger(level, message):
    color = 'white'
    if level == "INFO":
        color = 'cyan'
    elif level == "SUCCESS":
        color = 'green'
    elif level == "ERROR":
        color = 'red'
    elif level == "ACTION":
        color = 'blue'
    elif level == "PROGRESS" or level == "WARNING":
        color = 'yellow'
    elif level == "FINAL":
        color = 'magenta'
    logger.info(colored(f"SPARK: {level} - {message}", color))

# Set the environment variable for memory allocator
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

def initialize_cuda():
    cuda = ctypes.CDLL('libcuda.so')
    cuInit = cuda.cuInit
    cuInit.restype = ctypes.c_int
    cuInit.argtypes = [ctypes.c_uint]
    cuDeviceGetCount = cuda.cuDeviceGetCount
    cuDeviceGetCount.restype = ctypes.c_int
    cuDeviceGetCount.argtypes = [ctypes.POINTER(ctypes.c_int)]
    cuDeviceGet = cuda.cuDeviceGet
    cuDeviceGet.restype = ctypes.c_int
    cuDeviceGet.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]

    res = cuInit(0)
    if res != 0:
        raise RuntimeError("Failed to initialize CUDA")
    device_count = ctypes.c_int()
    res = cuDeviceGetCount(ctypes.byref(device_count))
    if res != 0:
        raise RuntimeError("Failed to get device count")
    devices = []
    for i in range(device_count.value):
        device = ctypes.c_int()
        res = cuDeviceGet(ctypes.byref(device), i)
        if res != 0:
            raise RuntimeError(f"Failed to get device {i}")
        devices.append(device.value)
    return devices

def allocate_gpu_memory(size):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_malloc = cuda.cuMemAlloc
    cuda_malloc.restype = ctypes.c_int
    cuda_malloc.argtypes = [ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_ulonglong]
    ptr = ctypes.c_ulonglong()
    res = cuda_malloc(ctypes.byref(ptr), size)
    if res != 0:
        raise RuntimeError("Failed to allocate GPU memory")
    return ptr.value

def free_gpu_memory(ptr):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_free = cuda.cuMemFree
    cuda_free.restype = ctypes.c_int
    cuda_free.argtypes = [ctypes.c_ulonglong]
    res = cuda_free(ptr)
    if res != 0:
        raise RuntimeError("Failed to free GPU memory")

# Check GPU memory usage
def check_gpu_memory():
    os.system('nvidia-smi')

# Initialize CUDA
cuda_devices = initialize_cuda()

# Check available GPU memory
check_gpu_memory()

# Ensure TensorFlow uses the GPU by setting memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")

# Check and print the number of GPUs available
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(f"Num GPUs Available: {num_gpus}")

# Perform a simple TensorFlow operation to verify GPU usage
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=tf.float32)
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=tf.float32)
    c = tf.matmul(a, b)
print(f"Result of matrix multiplication on GPU:\n{c.numpy()}")

def get_gpu_memory_usage():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        memory_used = int(result.stdout.decode('utf-8').strip())
        return memory_used
    except Exception as e:
        print(f"Failed to get GPU memory usage: {e}")
        return None

def freegpu():
    try:
        # Get GPU memory usage before freeing
        memory_before = get_gpu_memory_usage()

        result = subprocess.run(['sudo', 'fuser', '-v', '/dev/nvidia*'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in result.stdout.decode('utf-8').split('\n'):
            if '/dev/nvidia' in line:
                pid = int(line.split()[-1])
                os.kill(pid, 9)

        # Get GPU memory usage after freeing
        memory_after = get_gpu_memory_usage()

        if memory_before is not None and memory_after is not None:
            memory_freed = memory_before - memory_after
            print(f"GPU memory has been freed. {memory_freed} MB of GPU memory was released.")
        else:
            print("GPU memory has been freed.")
    except Exception as e:
        print(f"Failed to free GPU memory: {e}")

# Set the environment variable for XLA flags
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'

# ======================================================================================================
# ==============================[SETUP MIRROREDSTRATEGY WITHOUT NCCL]===================================
# ======================================================================================================

# ------------------------------------------------------------------------------------------------------
# Define the strategy with HierarchicalCopyAllReduce to avoid NCCL
# ------------------------------------------------------------------------------------------------------
strategy = tf.distribute.MirroredStrategy(
    devices=["/gpu:0", "/gpu:1"],
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()
)

print(f"Number of devices under strategy: {strategy.num_replicas_in_sync}")

NameError: name 'SparkSession' is not defined

# Univariate Analysis

This section performs univariate analysis on our dataset to understand the basic characteristics and quality of our data. The analysis includes:

- **Shape**: Dimensions of the dataset (rows x columns)
- **Data Types**: Type of each column (numeric, categorical, etc.)
- **Missing Values**: Count of null/NaN values per column
- **Duplicates**: Number of duplicate rows
- **Summary Statistics**: Basic statistics for numerical columns including:
  - Count
  - Mean
  - Standard Deviation 
  - Min/Max values
  - Quartile values
- **NaN Analysis**: Detailed breakdown of missing values by column with percentages

This analysis helps identify potential data quality issues and informs our preprocessing strategy.


In [ ]:
# Load the processed data for autoencoder training
original_data_train = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv', index_col=False, header=0)
original_data_test = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv', index_col=False, header=0)

original_data = pd.concat([original_data_train, original_data_test], ignore_index=True)

custom_logger("INFO", "ORIGINAL DATA - BEFORE PROCESSING")

# Univariate analysis
def univariate_analysis(df):
    print("Univariate Analysis\n" + "="*40)
    
    # Shape
    print(f"Shape:\n{df.shape}\n")
    
    # Data Types
    print(f"Data Types:\n{df.dtypes}\n")
    
    # Missing Values
    missing_values = df.isnull().sum()
    print(f"Missing Values:\n{missing_values}\n")
    
    # Duplicates
    print(f"Duplicates:\n{df.duplicated().sum()}\n")
    
    # Summary statistics for numerical columns
    print("Summary Statistics for Numerical Columns:\n")
    print(df.describe(include=[np.number]))
    
    # Detailed information about NaN values
    print("\nDetailed Information about NaN Values:\n")
    for col in df.columns:
        if df[col].isnull().any():
            print(f"Column '{col}' has {df[col].isnull().sum()} NaN values, which is {df[col].isnull().mean() * 100:.2f}% of the column.")

# Perform univariate analysis
univariate_analysis(original_data)

# Univariate Analysis Observations and Insights

### Observations:
1. **Shape**: The dataset contains 234,500 rows and 20 columns.
2. **Data Types**: The dataset includes a mix of numerical and categorical data types.
3. **Missing Values**: Several columns have a significant number of missing values:
    - 'Academic Pressure', 'CGPA', and 'Study Satisfaction' have 80.10% missing values.
    - 'Depression' has 40.00% missing values.
    - 'Profession' and 'Job Satisfaction' have around 20% missing values.
4. **Duplicates**: There are no duplicate rows in the dataset.
5. **Summary Statistics**:
    - The 'Age' column ranges from 18 to 60, with a mean of approximately 40.36.
    - 'Work/Study Hours' has a wide range from 0 to 12 hours, with a mean of 6.25 hours.
    - 'Depression' is a binary column with a mean of 0.18, indicating a low prevalence in the dataset.
6. **NaN Values**: Columns with significant NaN values may need imputation or removal depending on their importance.

### Insights:
- The high percentage of missing values in 'Academic Pressure', 'CGPA', and 'Study Satisfaction' suggests these columns may not be reliable for analysis without imputation.
- The dataset appears to be balanced in terms of age distribution, but further analysis is needed to understand the distribution of other categorical variables like 'Gender' and 'City'.
- The 'Depression' column, being binary, can be used as a target variable for classification tasks, but missing values need to be addressed.
- The dataset's lack of duplicates is beneficial for analysis, ensuring data integrity.

### Recommendations:
- Consider imputing missing values for critical columns or removing them if they are not essential.
- Explore the categorical variables further to understand their distribution and potential impact on the target variable.
- Investigate the relationship between 'Work/Study Hours' and 'Depression' to identify any potential correlations.

### Next Steps:
- Visualize the distribution of key variables to gain further insights.
- Perform bivariate analysis to explore relationships between variables.
- Address missing values through imputation or removal based on their significance to the analysis.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from pyspark.sql import SparkSession

def segment_populations(df):
    # Students: Have academic-related data
    students = df[df['CGPA'].notna()].copy()
    
    # Workers: Have work-related data but no academic data
    workers = df[
        (df['Work Pressure'].notna()) & 
        (df['Job Satisfaction'].notna()) & 
        (df['CGPA'].isna())
    ].copy()
    
    # Unemployed: No profession but also not students
    unemployed = df[
        (df['Profession'].isna()) & 
        (df['CGPA'].isna()) & 
        (df['Work Pressure'].isna())
    ].copy()
    
    return students, workers, unemployed

def prepare_features(df):
    # Convert categorical variables
    le = LabelEncoder()
    categorical_cols = ['Gender', 'City', 'Sleep Duration', 'Dietary Habits', 
                       'Have you ever had suicidal thoughts ?', 
                       'Family History of Mental Illness', 'Profession']
    
    # Create numeric DataFrame for correlation analysis
    df_numeric = df.copy()
    
    for col in categorical_cols:
        if col in df.columns and df[col].notna().any():
            df_numeric[col] = le.fit_transform(df[col].astype(str))
    
    # Convert remaining numeric columns
    numeric_cols = df_numeric.select_dtypes(include=['object']).columns
    for col in numeric_cols:
        try:
            df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
        except:
            print(f"Could not convert {col} to numeric")
    
    return df_numeric

def analyze_segment(df, segment_name):
    print(f"\nAnalyzing {segment_name} Population:")
    print(f"Population size: {len(df)}")
    
    # Basic statistics
    print("\nNumerical Features Summary:")
    print(df.describe())
    
    # Correlation matrix for numeric columns only
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        plt.figure(figsize=(12, 8))
        sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
        plt.title(f'Correlation Matrix - {segment_name}')
        plt.tight_layout()
        plt.show()
    
    # Distribution plots for key metrics
    plot_distributions(df, segment_name)

def plot_distributions(df, segment_name):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    n_cols = min(3, len(numeric_cols))
    if n_cols > 0:
        fig, axes = plt.subplots(1, n_cols, figsize=(15, 5))
        if n_cols == 1:
            axes = [axes]
        
        for i, col in enumerate(numeric_cols[:n_cols]):
            sns.histplot(data=df, x=col, ax=axes[i])
            axes[i].set_title(f'{col} Distribution')
        
        plt.suptitle(f'{segment_name} Population - Key Metrics Distribution')
        plt.tight_layout()
        plt.show()

# Main execution
students, workers, unemployed = segment_populations(original_data)

# Prepare features for each segment
students_prepared = prepare_features(students)
workers_prepared = prepare_features(workers)
unemployed_prepared = prepare_features(unemployed)

# Analyze each segment
analyze_segment(students_prepared, "Student")
analyze_segment(workers_prepared, "Worker")
analyze_segment(unemployed_prepared, "Unemployed")

# Plot depression rates across segments
def plot_depression_rates():
    plt.figure(figsize=(10, 6))
    
    segments = {
        'Students': students,
        'Workers': workers,
        'Unemployed': unemployed
    }
    
    depression_rates = []
    for name, segment in segments.items():
        rate = segment['Depression'].mean() * 100
        depression_rates.append({'Segment': name, 'Rate': rate})
    
    depression_df = pd.DataFrame(depression_rates)
    sns.barplot(x='Segment', y='Rate', data=depression_df)
    plt.title('Depression Rates Across Population Segments')
    plt.ylabel('Depression Rate (%)')
    plt.show()

plot_depression_rates()

# Convert pandas DataFrames to Spark DataFrames
students_spark = spark.createDataFrame(students)
workers_spark = spark.createDataFrame(workers) 
unemployed_spark = spark.createDataFrame(unemployed)

In [ ]:
# Create visualizations for missing values and data distributions
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set up the style
plt.style.use('default')
sns.set_palette("husl")

# Apply conditional data suppression based on segment criteria
custom_logger("ACTION", "Filtering data based on segment criteria...")

students_filtered = students[
    students['Academic Pressure'].notna() & 
    students['CGPA'].notna() & 
    students['Study Satisfaction'].notna()
].copy()

workers_filtered = workers[
    workers['Work Pressure'].notna() & 
    workers['Job Satisfaction'].notna() & 
    workers['Profession'].notna()
].copy()

unemployed_filtered = unemployed[
    (unemployed['CGPA'].isna()) &
    (unemployed['Work Pressure'].isna()) &
    (unemployed['Job Satisfaction'].isna()) &
    (unemployed['Profession'].isna())
].copy()

# Calculate and log data loss percentages
students_loss = (1 - len(students_filtered)/len(students)) * 100
workers_loss = (1 - len(workers_filtered)/len(workers)) * 100
unemployed_loss = (1 - len(unemployed_filtered)/len(unemployed)) * 100

custom_logger("INFO", f"Data loss after filtering:")
custom_logger("INFO", f"Students: {students_loss:.1f}%")
custom_logger("INFO", f"Workers: {workers_loss:.1f}%") 
custom_logger("INFO", f"Unemployed: {unemployed_loss:.1f}%")

# Create figure for missing values comparison
custom_logger("ACTION", "Generating visualization of missing values...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Missing values percentages by category after filtering
missing_vals = {
    'Students': {
        'Academic Pressure': (students_filtered['Academic Pressure'].isna().sum() / len(students_filtered)) * 100,
        'CGPA': (students_filtered['CGPA'].isna().sum() / len(students_filtered)) * 100,
        'Study Satisfaction': (students_filtered['Study Satisfaction'].isna().sum() / len(students_filtered)) * 100,
        'Depression': (students_filtered['Depression'].isna().sum() / len(students_filtered)) * 100
    },
    'Workers': {
        'Work Pressure': (workers_filtered['Work Pressure'].isna().sum() / len(workers_filtered)) * 100,
        'Job Satisfaction': (workers_filtered['Job Satisfaction'].isna().sum() / len(workers_filtered)) * 100,
        'Profession': (workers_filtered['Profession'].isna().sum() / len(workers_filtered)) * 100,
        'Depression': (workers_filtered['Depression'].isna().sum() / len(workers_filtered)) * 100
    },
    'Unemployed': {
        'Depression': (unemployed_filtered['Depression'].isna().sum() / len(unemployed_filtered)) * 100,
        'Financial Stress': (unemployed_filtered['Financial Stress'].isna().sum() / len(unemployed_filtered)) * 100
    }
}

# Plot missing values
x_pos = np.arange(len(missing_vals))
for i, (category, values) in enumerate(missing_vals.items()):
    ax1.bar(i, np.mean(list(values.values())), label=f'{category}\n(Avg: {np.mean(list(values.values())):.1f}%)')
    
ax1.set_title('Average Missing Values by Category (After Filtering)', fontsize=14)
ax1.set_ylabel('Percentage Missing (%)')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([k for k in missing_vals.keys()])

# Calculate and plot depression rates for filtered data
custom_logger("ACTION", "Calculating depression rates by category...")

depression_rates = {
    'Students': (students_filtered['Depression'].mean() * 100),
    'Workers': (workers_filtered['Depression'].mean() * 100),
    'Unemployed': (unemployed_filtered['Depression'].mean() * 100)
}

ax2.bar(depression_rates.keys(), depression_rates.values(), color='lightcoral')
ax2.set_title('Depression Rates by Category (After Filtering)', fontsize=14)
ax2.set_ylabel('Depression Rate (%)')

for i, v in enumerate(depression_rates.values()):
    ax2.text(i, v + 1, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.show()

# Convert filtered DataFrames to Spark DataFrames
custom_logger("ACTION", "Converting filtered data to Spark DataFrames...")

students_spark = spark.createDataFrame(students_filtered)
workers_spark = spark.createDataFrame(workers_filtered)
unemployed_spark = spark.createDataFrame(unemployed_filtered)

# Analyze missing data for each category
custom_logger("INFO", "Detailed Missing Data Analysis by Category:")

categories = {
    'Students': students_filtered,
    'Workers': workers_filtered, 
    'Unemployed': unemployed_filtered
}

# First check for duplicate IDs across categories
all_ids = []
for category, df in categories.items():
    if 'ID' in df.columns:
        all_ids.extend(df['ID'].tolist())

duplicate_ids = set([id for id in all_ids if all_ids.count(id) > 1])

if duplicate_ids:
    custom_logger("WARNING", f"Found {len(duplicate_ids)} duplicate IDs across categories")
    # Remove duplicates keeping first occurrence
    total_removed = 0
    for category, df in categories.items():
        if 'ID' in df.columns:
            initial_size = len(df)
            categories[category] = df[~df['ID'].isin(duplicate_ids)]
            removed_count = initial_size - len(categories[category])
            total_removed += removed_count
            custom_logger("INFO", f"Removed {removed_count} duplicate IDs from {category} category")
    custom_logger("INFO", f"Total of {total_removed} duplicate records removed across all dataframes")

else:
    custom_logger("SUCCESS", "No duplicate IDs found across categories")

# Now proceed with missing value analysis
for category, df in categories.items():
    custom_logger("INFO", f"\n{category} Category:")
    columns_to_drop = []
    for col in df.columns:
        null_count = df[col].isnull().sum()
        if null_count > 0:
            pct_missing = (null_count / len(df)) * 100
            custom_logger("WARNING" if pct_missing > 10 else "INFO", 
                         f"Column '{col}': {null_count} missing values ({pct_missing:.2f}%)")
            if pct_missing > 98:
                columns_to_drop.append(col)
                custom_logger("WARNING", f"Column '{col}' will be dropped due to {pct_missing:.2f}% missing values")
    
    if columns_to_drop:
        categories[category] = df.drop(columns=columns_to_drop)
        # Update the original filtered dataframes with dropped columns
        if category == 'Students':
            students_filtered = categories[category]
        elif category == 'Workers':
            workers_filtered = categories[category]
        elif category == 'Unemployed':
            unemployed_filtered = categories[category]
        custom_logger("INFO", f"Dropped {len(columns_to_drop)} columns from {category} category")

custom_logger("ACTION", "Performing univariate analysis...")

custom_logger("INFO", "Univariate Analysis Results:")

custom_logger("INFO", "Students:")
univariate_analysis(students_filtered)

custom_logger("INFO", "Workers:")
univariate_analysis(workers_filtered)

custom_logger("INFO", "Unemployed:")
univariate_analysis(unemployed_filtered)
